In [25]:
# read in the matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,abased,abasedand,abba,abel,abelto,abia,abide,abidein,abideth,abidethere,...,youyou,zabulon,zacchaeus,zacchaeuswanted,zacharias,zeal,zebedee,zebedees,zelotes,zorobabel
John,0,0,0,0,0,0,9,1,4,0,...,0,0,0,0,0,1,0,0,0,0
Luke,2,0,0,1,0,1,2,0,0,0,...,1,0,3,1,8,0,0,0,1,1
Mark,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,3,0,0,0
Matthew,0,1,0,0,1,0,1,0,0,0,...,0,1,0,0,1,0,3,1,0,0


In [26]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

In [27]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,John,Luke,Mark,Matthew
abased,0,2,0,0
abasedand,0,0,0,1
abba,0,0,1,0
abel,0,1,0,0
abelto,0,0,0,1


In [28]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [29]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [30]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.005*"people" + 0.005*"house" + 0.004*"day" + 0.004*"away" + 0.004*"let" + 0.003*"john" + 0.003*"saying" + 0.003*"took" + 0.003*"heard" + 0.003*"cast"'),
 (1,
  '0.019*"unto" + 0.006*"saith" + 0.005*"know" + 0.005*"world" + 0.005*"answered" + 0.005*"heaven" + 0.004*"sent" + 0.004*"did" + 0.004*"day" + 0.004*"jews"')]

In [31]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.032*"unto" + 0.011*"saith" + 0.009*"world" + 0.009*"know" + 0.008*"answered" + 0.007*"hath" + 0.007*"jews" + 0.006*"sent" + 0.005*"believe" + 0.005*"verily"'),
 (1,
  '0.000*"unto" + 0.000*"people" + 0.000*"know" + 0.000*"let" + 0.000*"men" + 0.000*"away" + 0.000*"day" + 0.000*"house" + 0.000*"sent" + 0.000*"took"'),
 (2,
  '0.005*"people" + 0.004*"day" + 0.004*"house" + 0.004*"took" + 0.004*"away" + 0.004*"heaven" + 0.004*"cast" + 0.004*"let" + 0.004*"did" + 0.004*"men"')]

In [32]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.001*"unto" + 0.000*"people" + 0.000*"day" + 0.000*"sent" + 0.000*"took" + 0.000*"answered" + 0.000*"john" + 0.000*"house" + 0.000*"know" + 0.000*"heard"'),
 (1,
  '0.005*"took" + 0.005*"heaven" + 0.005*"away" + 0.004*"cast" + 0.004*"unto" + 0.004*"did" + 0.004*"people" + 0.004*"men" + 0.004*"heard" + 0.004*"let"'),
 (2,
  '0.005*"house" + 0.005*"people" + 0.005*"day" + 0.004*"saying" + 0.004*"great" + 0.003*"let" + 0.003*"tell" + 0.003*"good" + 0.003*"called" + 0.003*"city"'),
 (3,
  '0.035*"unto" + 0.012*"saith" + 0.010*"world" + 0.009*"know" + 0.008*"answered" + 0.008*"hath" + 0.007*"jews" + 0.007*"sent" + 0.006*"believe" + 0.005*"verily"')]

In [33]:
# These topics could be better. Is unto a topic? no
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [34]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcripts
John,title the gospel of john for readers editor li...
Luke,title the gospel of luke for readers editor li...
Mark,title the gospel of mark for readers editor li...
Matthew,title the gospel of matthew for readers editor...


In [35]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcripts.apply(nouns))
data_nouns

,transcripts
John,gospel john readers editor lightheart release ...
Luke,gospel luke readers editor lightheart release ...
Mark,gospel mark readers editor lightheart release ...
Matthew,gospel matthew readers editor lightheart relea...


In [36]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['andif','himand','like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said', 'unto', 'saith', 'hath', 'thou', 'ye', 'thee', 'hast', 'shalt',
                 'sayest', 'cometh', 'woe']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=list(stop_words))
data_cvn = cvn.fit_transform(data_nouns.transcripts)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,abel,abide,abideth,abilene,ability,aboat,abode,abomination,aboutgalilee,aboutthee,...,youth,youyou,zabulon,zacchaeus,zacharias,zeal,zebedee,zebedees,zelotes,zorobabel
John,0,3,2,0,0,0,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Luke,1,1,0,1,0,0,1,0,0,1,...,1,1,0,2,8,0,0,0,1,1
Mark,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,3,0,0,0
Matthew,0,0,0,0,1,0,0,0,1,0,...,1,0,1,0,1,0,3,1,0,0


In [37]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [38]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.002*"jesus" + 0.001*"son" + 0.001*"man" + 0.001*"god" + 0.001*"things" + 0.001*"father" + 0.001*"disciples" + 0.001*"day" + 0.001*"lord" + 0.001*"world"'),
 (1,
  '0.048*"jesus" + 0.027*"man" + 0.019*"son" + 0.014*"god" + 0.013*"things" + 0.012*"father" + 0.011*"disciples" + 0.011*"lord" + 0.008*"day" + 0.007*"kingdom"')]

In [39]:
# looking better
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.050*"jesus" + 0.028*"man" + 0.020*"son" + 0.015*"god" + 0.014*"things" + 0.013*"father" + 0.012*"disciples" + 0.012*"lord" + 0.008*"day" + 0.007*"kingdom"'),
 (1,
  '0.002*"jesus" + 0.001*"man" + 0.001*"son" + 0.001*"god" + 0.001*"disciples" + 0.001*"lord" + 0.001*"father" + 0.001*"things" + 0.000*"men" + 0.000*"house"'),
 (2,
  '0.002*"jesus" + 0.001*"son" + 0.001*"man" + 0.001*"god" + 0.001*"lord" + 0.001*"things" + 0.001*"day" + 0.001*"father" + 0.001*"house" + 0.000*"disciples"')]

In [40]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.001*"jesus" + 0.001*"man" + 0.001*"son" + 0.001*"god" + 0.001*"things" + 0.001*"disciples" + 0.001*"lord" + 0.001*"father" + 0.000*"day" + 0.000*"house"'),
 (1,
  '0.041*"jesus" + 0.025*"man" + 0.022*"son" + 0.014*"lord" + 0.014*"god" + 0.013*"things" + 0.009*"disciples" + 0.009*"kingdom" + 0.009*"day" + 0.008*"father"'),
 (2,
  '0.051*"jesus" + 0.026*"man" + 0.025*"father" + 0.018*"world" + 0.015*"son" + 0.015*"god" + 0.014*"things" + 0.014*"jews" + 0.012*"disciples" + 0.009*"life"'),
 (3,
  '0.052*"jesus" + 0.028*"man" + 0.012*"disciples" + 0.011*"son" + 0.010*"things" + 0.010*"god" + 0.006*"house" + 0.006*"hand" + 0.006*"kingdom" + 0.006*"way"')]

In [41]:
# Let's create a function to pull out nouns and adjectives from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [42]:
# Apply the nouns function to the transcripts to filter only on nouns and adjectives
data_nouns_adj = pd.DataFrame(data_clean.transcripts.apply(nouns_adj))
data_nouns_adj

,transcripts
John,gospel john readers editor lightheart release ...
Luke,gospel luke readers editor lightheart release ...
Mark,gospel mark readers editor lightheart release ...
Matthew,gospel matthew readers editor lightheart relea...


In [43]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=list(stop_words), max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcripts)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,abel,abia,abide,abideth,abilene,ability,able,ableto,aboat,abode,...,youwill,youyou,zabulon,zacchaeus,zacharias,zeal,zebedee,zebedees,zelotes,zorobabel
John,0,0,3,2,0,0,1,1,0,2,...,0,0,0,0,0,1,0,0,0,0
Luke,1,1,1,0,1,0,6,0,0,1,...,1,1,0,2,8,0,0,0,1,1
Mark,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,3,0,0,0
Matthew,0,0,0,0,0,1,9,0,0,0,...,0,0,1,0,1,0,3,1,0,0


In [44]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [45]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.005*"light" + 0.004*"law" + 0.003*"devils" + 0.003*"eye" + 0.003*"angel" + 0.002*"generation" + 0.002*"disciple" + 0.002*"faith" + 0.002*"works" + 0.002*"worthy"'),
 (1,
  '0.003*"james" + 0.002*"devils" + 0.002*"herod" + 0.002*"unclean" + 0.002*"elders" + 0.002*"cast" + 0.002*"damsel" + 0.002*"soul" + 0.002*"false" + 0.002*"heed"')]

In [46]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.004*"eye" + 0.003*"field" + 0.003*"talents" + 0.003*"light" + 0.003*"devils" + 0.003*"faith" + 0.003*"elders" + 0.003*"generation" + 0.003*"hypocrites" + 0.003*"herod"'),
 (1,
  '0.003*"james" + 0.003*"devils" + 0.003*"herod" + 0.003*"unclean" + 0.003*"elders" + 0.002*"cast" + 0.002*"damsel" + 0.002*"swine" + 0.002*"soul" + 0.002*"false"'),
 (2,
  '0.006*"light" + 0.005*"law" + 0.003*"disciple" + 0.003*"fathers" + 0.003*"devils" + 0.003*"angel" + 0.003*"rich" + 0.003*"joy" + 0.002*"works" + 0.002*"judge"')]

In [47]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.004*"james" + 0.003*"devils" + 0.003*"herod" + 0.003*"unclean" + 0.003*"elders" + 0.003*"cast" + 0.003*"damsel" + 0.002*"false" + 0.002*"soul" + 0.002*"heed"'),
 (1,
  '0.004*"devils" + 0.004*"light" + 0.004*"eye" + 0.004*"generation" + 0.003*"angel" + 0.003*"faith" + 0.003*"herod" + 0.003*"rich" + 0.003*"worthy" + 0.003*"field"'),
 (2,
  '0.000*"law" + 0.000*"light" + 0.000*"faith" + 0.000*"herod" + 0.000*"worthy" + 0.000*"devils" + 0.000*"rich" + 0.000*"judge" + 0.000*"joy" + 0.000*"disciple"'),
 (3,
  '0.008*"light" + 0.006*"law" + 0.006*"disciple" + 0.005*"works" + 0.005*"sin" + 0.004*"fathers" + 0.004*"philip" + 0.003*"record" + 0.003*"miracles" + 0.003*"love"')]

In [48]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.004*"eye" + 0.004*"field" + 0.003*"talents" + 0.003*"elders" + 0.003*"faith" + 0.003*"generation" + 0.003*"devils" + 0.003*"hypocrites" + 0.003*"light" + 0.003*"prison"'),
 (1,
  '0.000*"light" + 0.000*"law" + 0.000*"works" + 0.000*"shepherd" + 0.000*"branch" + 0.000*"honour" + 0.000*"sponge" + 0.000*"deny" + 0.000*"marvellous" + 0.000*"tabernacles"'),
 (2,
  '0.005*"devils" + 0.004*"rich" + 0.004*"herod" + 0.003*"james" + 0.003*"generation" + 0.003*"angel" + 0.003*"unclean" + 0.003*"faith" + 0.002*"cast" + 0.002*"women"'),
 (3,
  '0.008*"light" + 0.006*"law" + 0.006*"disciple" + 0.005*"works" + 0.005*"sin" + 0.004*"fathers" + 0.004*"philip" + 0.003*"love" + 0.003*"record" + 0.003*"miracles"')]

In [49]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(3, 'John'), (2, 'Luke'), (2, 'Mark'), (0, 'Matthew')]